In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk import word_tokenize # Tokenizador


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cristianmurillo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/cristianmurillo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cristianmurillo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df = pd.read_csv('/Users/cristianmurillo/Documents/Data scientis /NLP/Algoritmos de Clasificacion de texto/datasets/email/csv/spam-apache.csv',names=['clase','mensaje'])
df['tokens'] = df['mensaje'].apply(lambda x: word_tokenize(x))
df.head()

,clase,mensaje,tokens
0,-1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...","[<, !, DOCTYPE, HTML, PUBLIC, ``, -//W3C//DTD,..."
1,1,> Russell Turpin:\n> > That depends on how the...,"[>, Russell, Turpin, :, >, >, That, depends, o..."
2,-1,Help wanted. We are a 14 year old fortune 500...,"[Help, wanted, ., We, are, a, 14, year, old, f..."
3,-1,Request A Free No Obligation Consultation!\nAc...,"[Request, A, Free, No, Obligation, Consultatio..."
4,1,Is there a way to look for a particular file o...,"[Is, there, a, way, to, look, for, a, particul..."


In [4]:
# frecuencia de palabras en la columna tokens
from nltk.probability import FreqDist

all_words = [word for tokens in df['tokens'].values for word in tokens] # lista de palabras
all_words = nltk.FreqDist(all_words) # frecuencia de palabras
all_words.most_common(5) # 15 palabras mas comunes



[(',', 2173), ('.', 2171), ('the', 1967), ('>', 1787), ('--', 1611)]

In [5]:
# tupla de palabras y frecuencia
bigrams = [bigrams for tokens in df['tokens'].values for bigrams in nltk.bigrams(tokens)]
all_bigrams = nltk.FreqDist(bigrams)
all_bigrams.most_common(5) # 15 bigramas mas comunes


[(('--', '--'), 1416),
 (('*', '*'), 969),
 (('<', 'BR'), 494),
 (('BR', '>'), 494),
 (('>', '>'), 404)]

In [6]:
# atributos del mensaje
def atributos_mensaje(mensaje):
    words = set(word_tokenize(mensaje)) # tokenizar el mensaje y convertirlo en un conjunto de palabras unicas set() para eliminar duplicados
    bigrams = nltk.bigrams(mensaje)
    atributos = {}
    #for word in all_words:
    #    atributos['contain({})'.format(word)]= (word in words)
    for word,bigram in zip(all_words,all_bigrams):
        atributos['contain({})'.format(word)]= (word in words)
        atributos['contain({})'.format(bigram)]= (bigram in bigrams)    
    return atributos
    

# atributos del mensaje

x = atributos_mensaje(df['mensaje'][0])
x.items()

dict_items([('contain(,)', True), ("contain(('--', '--'))", False), ('contain(.)', True), ("contain(('*', '*'))", False), ('contain(the)', True), ("contain(('<', 'BR'))", False), ('contain(>)', True), ("contain(('BR', '>'))", False), ('contain(--)', True), ("contain(('>', '>'))", False), ('contain(to)', True), ("contain(('http', ':'))", False), ('contain(:)', True), ("contain(('#', '#'))", False), ('contain(*)', True), ("contain((':', ':'))", False), ('contain(and)', True), ("contain(('%', '31'))", False), ('contain(of)', True), ("contain(('31', '%'))", False), ('contain(a)', True), ("contain(('>', '<'))", False), ('contain(you)', True), ("contain(('30', '%'))", False), ('contain(in)', True), ("contain(('%', '30'))", False), ('contain(I)', True), ("contain(('of', 'the'))", False), ('contain(<)', True), ("contain(('!', '!'))", False), ('contain(!)', True), ("contain((',', 'and'))", False), ('contain(%)', True), ("contain(('in', 'the'))", False), ('contain(for)', True), ("contain(('$', '

In [7]:
fset = [(atributos_mensaje(mensaje),clase) for mensaje,clase in zip(df['mensaje'],df['clase'])] # lista de tuplas
#fset split train y test
import random
random.shuffle(fset)

train, test = fset[200:], fset[:200]


In [11]:
g = pd.DataFrame(fset)
g


,0,1
0,"{'contain(,)': True, 'contain(('--', '--'))': ...",-1
1,"{'contain(,)': True, 'contain(('--', '--'))': ...",1
2,"{'contain(,)': True, 'contain(('--', '--'))': ...",-1
3,"{'contain(,)': True, 'contain(('--', '--'))': ...",-1
4,"{'contain(,)': True, 'contain(('--', '--'))': ...",1
...,...,...
245,"{'contain(,)': True, 'contain(('--', '--'))': ...",1
246,"{'contain(,)': True, 'contain(('--', '--'))': ...",1
247,"{'contain(,)': True, 'contain(('--', '--'))': ...",1
248,"{'contain(,)': False, 'contain(('--', '--'))':...",-1


In [26]:
classifier = nltk.NaiveBayesClassifier.train(train) # entrenamiento

In [27]:
print(nltk.classify.accuracy(classifier, test)) # precision

0.905
